<a href="https://colab.research.google.com/github/Luke-zm/coursera_learning/blob/main/udemy_pyomo/pyomo_ip_tut1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integer Programming

Integer programming is very similar to linear programming.  
The objective, and constraints have to be linear.  
The variables must be integer, instead of any real number in linear programming.  
$$  
max~ z = f(x)  
\\s.t  
\\g_1(x) \leq 0  
\\g_2(x) = 0  
\\ x \in Z^n  
$$  
Where:  
max(Z) = f(x) is linear objective function  
g1 and g2 are linear constraint functions  
and x is a element of real integer number  
Integer programming is when some, or all of the variables are restricted to be integers.

If all variables are integers ==> Integer Programming (IP)    
Some variables are integers ==> Mixed Interger programming (MIP)  
Some variables are intergers, and Linear Problem ==> Mixed Integer Linear Programming (MILP)   
Usually solved via things like branch and bound etc.  

## Problem 1
Colthes company capable of manufacturing 3 types of clothing: shirt, shorts and pants. Manufacturing each type of clothing requires appropriate types of machinery to be available. Machinaries are rented.  
Shirt: \$200   
Shorts: \$150    
Pants: \$100  
Clothes and time are also needed, as shown in the table below:  

|Type |Sales Price(\$)| Cost($)|Labour (hr)| Cloth (sqyd)|   
|-----|---------------|--------|-----------|-------------|   
|Shirt| 12            | 6      | 3         | 4           |  
|Short| 8             | 4      | 2         | 3           |
|Pants| 15            | 8      | 6         | 4           |   

There are only 150 hoursof labour and 160 sqyd of clothes available each week.  
Formulate an IP solution where profit will be maximised.  

<strong>Constraint 1: </strong>At most 150 hrs of labour used each week.  
<strong>Constraint 2: </strong>At most 160 sqyd of cloth used each week.  


## Mathematical formulation  
Let the index for the 3 products be i, where i = Shirt, Shorts, Pants.  
Let Sals Price be S, Cost be C, Labour be L, Cloth be F (Fabric), Machine rent cost be M.  
Let Profit be P, where P = S - C.  
Let the number of each type of product made be x.
Let Rent be R, where renting the corresponding machine will be 1, not renting will be 0.   

$$
\\ max ((P_i\times x_i) - R_i \times  M_i)  
\\s.t  
\\sum(x_i\times L_i) \leq 150  
\\sum(x_i\times F_i) \leq 160
\\ x_i \geq 0
\\ R_i = 0~ or~ 1
$$

However, as we are noe using binary variables inour problem...   
It is natural that we will need to find a way to link our R with production.  
To link this 2 up, we usually use the big M method.  
Therefore, we have an additional constraint.  
$$
\\x_i \leq M_i \times R_i
$$  

The big M method is used to link the binary variable in this case to the number of each type of products produced.  
The idea is simple.  
When R is 0, then x, the number of product produced, must be 0.  
When R is 1, then R multiplied by big M, will have to be bigger than that type of product produced.  
This basically helps to enforce that.  
From my personal experience, big M should not be too small, nor should it be too big.  
In this case, the max number of any 1 type of product is 160/3=53.3.  
So a small big M can be 54.  
Why keep big M small?   
My guess is to reduce the search space.  

In [ ]:
!pip install pyomo
!pip install cplex -q

In [ ]:
!apt-get install -y -qq glpk-utils

In [ ]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [ ]:
# Set the index
clothes = ['shirt', 'shorts', 'pants']

# Set the Parameters
sell_price = {'shirt':12, 'shorts':8, 'pants':15}
prod_cost = {'shirt':6, 'shorts':4, 'pants':8}
labour_hr = {'shirt':3, 'shorts':2, 'pants':6}
cloth_used = {'shirt':4, 'shorts':3, 'pants':4}
rental_cost = {'shirt':200, 'shorts':150, 'pants':100}

# Set the constraints
available_labour = 150
available_mat = 160

# Big M
big_M = {cloth: min((available_mat//i, available_labour//j))
        for i, j, cloth in
         zip(cloth_used.values(), labour_hr.values(), clothes)}
big_M

{'shirt': 40, 'shorts': 53, 'pants': 25}

In [ ]:
# Build the model accordingly
model = pyo.ConcreteModel()

# Create indexing sets
model.clothes = pyo.Set(initialize=clothes)
cloth_idx = model.clothes

# Create the decision varaibles on number of pieces of clothes to make
model.num_prod = pyo.Var(cloth_idx, within=pyo.NonNegativeIntegers)
num_prod = model.num_prod

model.rent = pyo.Var(model.clothes, within=pyo.NonNegativeIntegers)
rent = model.rent


In [ ]:
# Define the model parameters
model.sell_price = pyo.Param(cloth_idx, initialize=sell_price)
model.prod_cost = pyo.Param(cloth_idx, initialize=prod_cost)
model.lab_hr = pyo.Param(cloth_idx, initialize=labour_hr)
model.mat_used = pyo.Param(cloth_idx, initialize=cloth_used)
model.rent_cost = pyo.Param(cloth_idx, initialize=rental_cost)
model.big_M_val = pyo.Param(cloth_idx, initialize=big_M)

In [ ]:
# Objective function
def max_profit(model, cloth_idx):
  profit = sum(model.num_prod[cloth]*(model.sell_price[cloth] - model.prod_cost[cloth]) for cloth in model.clothes) - sum(model.rent[cloth]*model.rent_cost[cloth] for cloth in model.clothes)
  return profit
model.obj=pyo.Objective(expr=max_profit, sense=pyo.maximize)

In [ ]:
# Add in labour constriants
def labour_const(model, cloth):
  return sum(model.lab_hr[cloth] * model.num_prod[cloth] for cloth in model.clothes) <= available_labour
model.lab_const = pyo.Constraint(model.clothes, rule=labour_const)

# Add in material constraints
def material_const(model, cloth):
  return sum(model.mat_used[cloth]*model.num_prod[cloth] for cloth in model.clothes) <= available_mat
model.mat_const = pyo.Constraint(model.clothes, rule=material_const)

# Add in big M constraints
def big_M_const(model, cloth):
  return model.num_prod[cloth] <= model.big_M_val[cloth] * model.rent[cloth]
model.big_M_const = pyo.ConstraintList()
for cloth in model.clothes:
  model.big_M_const.add(
      expr=big_M_const(model, cloth)
  )

In [ ]:
Solver = SolverFactory('cplex_direct')
results =Solver.solve(model)

print(results)


Problem: 
- Name: 
  Lower bound: 75.00000000000031
  Upper bound: 75.00000000000031
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 6
  Number of binary variables: 0
  Number of integer variables: 6
  Number of continuous variables: 0
  Number of nonzeros: None
  Sense: -1
Solver: 
- Name: CPLEX 22.1.1.0
  Status: ok
  Wallclock time: 0.04101061820983887
  Termination condition: optimal
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [ ]:
print('Objective func = ', model.obj())

Objective func =  75.00000000000031


In [ ]:
for cloth in model.clothes:
  print(f"{model.num_prod[cloth]()} pieces of {cloth} should be produced")

0.0 pieces of shirt should be produced
1.2789769243681803e-13 pieces of shorts should be produced
24.999999999999947 pieces of pants should be produced


In [ ]:
Solver_glpk = SolverFactory('glpk')
results_glpk =Solver_glpk.solve(model)

print(results_glpk)

print('Objective func = ', model.obj())

for cloth in model.clothes:
  print(f"{model.num_prod[cloth]()} pieces of {cloth} should be produced")


Problem: 
- Name: unknown
  Lower bound: 75.0
  Upper bound: 75.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 6
  Number of nonzeros: 24
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 3
      Number of created subproblems: 3
  Error rc: 0
  Time: 0.006719350814819336
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objective func =  75.0
0.0 pieces of shirt should be produced
0.0 pieces of shorts should be produced
25.0 pieces of pants should be produced
